In [4]:
#Capstone
#Group 1
import webbrowser
import requests
import flask
import sqlite3
import json
import csv
from bs4 import BeautifulSoup
from urllib.request import urlopen

class data:
    def __init__(self):
        #create table if not exists Jobs (jobTitle text, passWord text, userType text)
        #info given by user
        self.location="none"
        self.jobType="no type"
        self.skills=[]
        self.exp="none"
        self.edu="none"

        #info given by api
        self.age=[]#how old is the job listing
        self.jobLst=[]
        self.jobDist=[]
        self.jobLocation="none"
        self.company="none"
        self.listing=[]
        
        # keywords used in skill identification
        self.keyWordSkills = ['python', 'java', 'c++', 'sql', 'manage', 'javascript', 'linux', 'team', 'problem solving', 'front end', 'back end', 'html', 'css','json', 'xml','api', 'linux', 'nodejs', 'c#', 'spark', 'sas', 'matlab', 'excel', 'spark', 'hadoop', 'azure', 'spss', 'git', 'aws']
        self.keyWordEdu = ['masters', 'bachelors', "master's", "bachelor's", 'phd', 'undergrad', 'graduate', 'undergraduate', 'ged', "graduate's", "undergraduate's", "associate's", 'doctoral']
        self.aiKeys = ['ai', 'a.i.', 'artificial intelligence', 'artificial']
        self.dlKeys= ['deep learning', 'neural networks', 'big data', 'deep', 'statistics']
        self.mlKeys = ['data mining', 'machine learning', 'cnn', 'rbm', 
          'machine', 'natural language', 'regression', 'fault diagnosis', 'intrusion detection']
        self.seKeys = ['software engineer', 'software development','code']

    def __repr__(self):
        return self.exp
    
    def port_to_csv(self):
        inpsql3 = sqlite3.connect('Flask_Jade_Sample/TestFlaskJadeWeb/Users.db')
        sql3_cursor = inpsql3.cursor()
        sql3_cursor.execute('SELECT * FROM JOBS')
        with open('output.csv','w') as out_csv_file:
            csv_out = csv.writer(out_csv_file)
            # write header                        
            csv_out.writerow([d[0] for d in sql3_cursor.description])
            # write data                          
            for result in sql3_cursor:
                csv_out.writerow(result)
        inpsql3.close()
    
    # TODO: remove function?
    # sends the data allocated to the database
    def testing(self):
        conn = sqlite3.connect("Flask_Jade_Sample/TestFlaskJadeWeb/Users.db")
        cursor = conn.cursor()
        
        # create table if not exists
        # columns used for the primary key implicitly cannot be null
        # columns skills and education are comma separated string representations of lists
        table_query = """create table if not exists JOBS
                            (location text, company text, datePosted text, postUrl text, 
                            jobType text, jobTitle text, jobDes text, jobApp text, salary text,
                            skills text, category text, education text,
                            PRIMARY KEY (company, jobTitle, location))"""
        cursor.execute(table_query)
        conn.commit()
        
        for i in range(len(self.jobLst)):
            insert_query = """insert or ignore into JOBS (location, company, datePosted, postUrl, 
                                                jobType, jobTitle, jobDes, jobApp, salary, skills,
                                                category, education) 
                                    VALUES (?,?,?,?,?,?,?,?,?,?,?,?)"""
            
            #print(self.jobLst[i]['Skills'])
            data_tuples = (self.jobLst[i]['Location'], self.jobLst[i]['Company'], 
                           self.jobLst[i]['Time-Posted'], self.jobLst[i]['Page-Addr'], 
                           self.jobLst[i]['Contract-Type'], self.jobLst[i]['Title'], 
                           self.jobLst[i]['Desc'], self.jobLst[i]['Apply-To'], self.jobLst[i]['Salary'],
                           lst_to_str(self.jobLst[i]['Skills']), self.jobLst[i]['Category'], lst_to_str(self.jobLst[i]['Education']))

            cursor.execute(insert_query, data_tuples)

        conn.commit()
        cursor.close()
        conn.close()

    def creatUsersTable(self):
        conn = sqlite3.connect("Flask_Jade_Sample/TestFlaskJadeWeb/Users.db")
        cursor = conn.cursor()
        table_query = """create table if not exists Users
                            (userName text, passWord text, userType text,
                            Points integer, email text, skills text, 
                            bio text, Bookmarks text, location text,
                            PRIMARY KEY (userName))"""
        cursor.execute(table_query)
        conn.commit()
        cursor.close()
        conn.close()
        
    def checkIfUserExist(self, user):
        conn = sqlite3.connect("Flask_Jade_Sample/TestFlaskJadeWeb/Users.db")
        cursor = conn.cursor()
        
        select_query = 'select * from Users where userName = ?'
        cursor.execute(select_query, (user,))
        records = cursor.fetchall()
        
        return True if len(records) == 1 else False
        
    def destroyUsersTable(self):
        conn = sqlite3.connect("Flask_Jade_Sample/TestFlaskJadeWeb/Users.db")
        cursor = conn.cursor()
        table_query = 'DROP TABLE Users'
        cursor.execute(table_query)
        cursor.close()
        conn.close()
        
    # searches backend for whatever search term
    def searchJobs(self, term):
        results = []
        
        conn = sqlite3.connect("Flask_Jade_Sample/TestFlaskJadeWeb/Users.db")
        cursor = conn.cursor()
        select_query = """select * from JOBS """ # change JOBS to whatever table you want to see
        
        cursor.execute(select_query)
        records = cursor.fetchall()
        cursor.close()
        conn.close()
        
        for i in range(len(records)):
            if term in records[i]:
                results.append(records[i])
                #return records[i] # TODO: remove to return multiple results
        
        return results
        
    # returns tuple list of all records in jobs table sorted by the job title in ascending order
    def getAllJobs(self):
        conn = sqlite3.connect("Flask_Jade_Sample/TestFlaskJadeWeb/Users.db")
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        select_query = """select * from Jobs order by jobTitle ASC"""
        cursor.execute(select_query)
        records = cursor.fetchall()
        cursor.close()
        conn.close()
        return [dict(row) for row in records]
    
    def getAllUsers(self):
        conn = sqlite3.connect("Flask_Jade_Sample/TestFlaskJadeWeb/Users.db")
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        select_query = """select * from Users"""
        cursor.execute(select_query)
        records = cursor.fetchall()
        cursor.close()
        conn.close()
        return [dict(row) for row in records]

    # returns tuple list of jobs starting from the offset and getting as many as amount
    # 0 based indexing means offset at 1 will start at second index
    def getNJobs(self, offset, amt):
        conn = sqlite3.connect("Flask_Jade_Sample/TestFlaskJadeWeb/Users.db")
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        
        select_query = """select * from Jobs order by jobTitle ASC"""
        cursor.execute(select_query)
        records = cursor.fetchall()
        cursor.close()
        conn.close()
        
        return [[records.index(row), dict(row)] for row in records[offset:offset+amt]]
    
    # modification of getNJobs
    # similar design but will search the specified column (col) for the search term (term)
    # return is the same
    # col MUST BE a column header
    def getNJobsByQuery (self, term, col, offset, amt):
        conn = sqlite3.connect("Flask_Jade_Sample/TestFlaskJadeWeb/Users.db")
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        
        if col[1] == 'None' and col[2] == 'None':
            select_query = 'select * from Jobs where '+ col+ ' like ? order by jobTitle ASC'
            cursor.execute(select_query, ('%'+term+'%',)) # pattern matching
        
        elif not col[1] == 'None' and col[2] == 'None':
            select_query = 'select * from Jobs where '+ col[0] + ' like ? and ' + col[1] + ' like ? ' + ' order by jobTitle ASC'
            cursor.execute(select_query, ('%'+term[0]+'%', '%'+term[1]+'%')) # pattern matching
        
        elif not col[1] == 'None' and not col[2] == 'None':
            select_query = 'select * from Jobs where '+ col[0] + ' like ? and ' + col[1] + ' like ? and ' + col[2] + ' like ? ' + ' order by jobTitle ASC'
            cursor.execute(select_query, ('%'+term[0]+'%', '%'+term[1]+'%', '%'+term[2]+'%',)) # pattern matching
        
        records = cursor.fetchall()
        cursor.close()
        conn.close()
        
        return [[records.index(row), dict(row)] for row in records[offset:offset+amt]]
    
    # returns a single job as a dictionary based on its index when sorting by title
    def getNthJob(self, n):
        conn = sqlite3.connect("Flask_Jade_Sample/TestFlaskJadeWeb/Users.db")
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        
        select_query = """select * from Jobs order by jobTitle ASC"""
        cursor.execute(select_query)
        records = cursor.fetchall()
        cursor.close()
        conn.close()
        
        return [dict(row) for row in records][n]
    
    def getUserByName(self, name):
        conn = sqlite3.connect("Flask_Jade_Sample/TestFlaskJadeWeb/Users.db")
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        
        select_query = 'select * from Users where userName = ?'
        cursor.execute(select_query, (name,))
        records = cursor.fetchall()
        if len(records) == 1:
            return [dict(row) for row in records][0]
        else:
            print('-- ERROR: COULD NOT FIND USER: ' + name + ' --')
            return None
    
    # deletes the jobs table entirely
    def destroyJobs(self):
        conn = sqlite3.connect("Flask_Jade_Sample/TestFlaskJadeWeb/Users.db")
        cursor = conn.cursor()
        table_query = 'DROP TABLE JOBS'
        cursor.execute(table_query)
        cursor.close()
        conn.close()
    
###CAN CHANGE TO RUN LINKS THROUGH METHODS (have links as method parameters)

    #returns list of links of positions without location specification 
    def onlylinks(self):
        linklist = []
        url = 'https://jobs.github.com/positions'
        source = requests.get(url).text
        soup = BeautifulSoup(source, 'lxml')

        for link in soup.find_all('a'):
            links = link.get('href')
            if "http" in links and '/positions/' in links:
               #print (links)
                linklist.append(link['href'])
        return linklist
            
    # searches multiple pages and the given location
    # returns list of all jobs found
    # also returns links and other info
    def hubJobs(self, jobX, loc, numPages):
        print('entering github jobs')
        multPageLocJobsWL = [] # will be populated with job dictionaries
        for i in range (0, numPages): # iterating over pages until numPages
            loc = '' if loc == '' else ('&location=' + loc.replace(' ', '+'))
            jobX = '' if jobX == '' else ('&description=' + jobX.replace(' ', '+'))
            finalURL = 'https://jobs.github.com/positions?utf8=✓' + loc + '&page=' + str(i) + jobX
            #finalURL = url + loc # including location in filters
            
            source = requests.get(finalURL).text
            soup = BeautifulSoup(source, 'lxml')
            
            for job in soup.find_all('tr', {'class':'job'} ): # iterating over individual job data
                tmp = job.text.strip().split('\n')
                jb = {}
                for x in range (0, len(tmp)):
                    y = tmp[x].strip()
                    if len(y) > 1 and not "\t" in y:
                        if x == 0:
                            jb['Title'] = y
                        
                        elif x == 2:
                            jb['Company'] = y
                        
                        elif x == 4:
                            jb['Contract-Type'] = y
                        
                        elif x == 7:
                            jb['Location'] = y
                        
                        elif x == 8:
                            jb['Time-Posted'] = y
                        
                jb['Salary'] = 'N/A' # salary not listed on github
                jobMeta = self.getPageMeta (job.find('td', {'class':'title'}).find('h4').find('a')['href']) # gets job info (applyto link and skills)
                jb['Apply-To'] = jobMeta[0]
                jb['Skills'] = jobMeta[1]
                jb['Desc'] = jobMeta[2]
                jb['Page-Addr'] = jobMeta[3]
                jb['Education'] = jobMeta[4]
                jb['Category'] = jobMeta[5]
                
                multPageLocJobsWL.append(jb)
                
        print('github jobs length ', len(multPageLocJobsWL))
        return multPageLocJobsWL
            
    # TODO
    #going to be a list of dictionaries ***TBA
    #for now prints out several dictionaries for each job post on the page (50)
    def create(self, job = '', loc = '', numPages = 1):
        self.jobLst = self.hubJobs (job, loc, numPages) + self.indeedJobs(job, loc, numPages)
        #self.jobLst += self.indeedJobs(job, loc, numPages)
        self.testing()
        print("create done")

    # returns 4 pieces of the specified page
    #   [string:applyTo_link, list:skills, string:description, string:page_url, list:education]
    def getPageMeta(self, url):
        newSrc = requests.get(url).text
        newSoup = BeautifulSoup(newSrc, 'lxml')
        
        # apply-to link
        jobLink = ''
        try:
            jobLink = newSoup.find('div', {'class':'highlighted'}).find('a')['href']
        except:
            jobLink = url
        
        foundSkillsList = []
        foundEduList = []

        # parsing summary
        summary = newSoup.find('div', class_=['column main', 'jobsearch-jobDescriptionText'])
        sumtext = summary.text if not summary == None else 'N/A'
        sumtext1 = sumtext.lower()
        sumList = sumtext1.split()
        
        # education
        for i in self.keyWordEdu:
            if i in sumList and not i in foundEduList:
                foundEduList.append(i)
                
        # skills
        for i in self.keyWordSkills:
            if i in sumList and not i in foundSkillsList:
                foundSkillsList.append(i)
        
        # primitive text classification
        # sums up occurunces of keywords and then appends the category tag associated with the highest count
        cat = ''
        aiCNT = 0
        dlCNT = 0
        mlCNT = 0
        seCNT = 0
        otherCNT = 0
        for x in sumList:
            if x in self.aiKeys:
                aiCNT += 1
                continue
            elif x in self.dlKeys:
                dlCNT += 1
                continue
            elif x in self.mlKeys:
                mlCNT += 1
                continue
            elif x in self.seKeys:
                seCNT += 1
                continue
                
        mx = max(aiCNT, dlCNT, mlCNT, seCNT, otherCNT)
        if aiCNT == mx:
            cat = 'Artificial Intelligence'
        elif dlCNT == mx:
            cat = 'Deep Learning'
        elif mlCNT == mx:
            cat = 'Machine Learning'
        elif seCNT == mx:
            cat = 'Software Engineer'
        elif otherCNT == mx: # consider difference of counts?
            cat = 'Other'
        
        return [jobLink, foundSkillsList, sumtext, url, foundEduList, cat]
    
    def addCol(self):
        conn = sqlite3.connect("Flask_Jade_Sample/TestFlaskJadeWeb/Users.db")
        cursor = conn.cursor()
        addColumn = "ALTER TABLE Users ADD COLUMN email text"
        cursor.execute(addColumn)
        cursor.close()
        conn.close()
        return 

    def gamePoints(self,user,points):
        conn = sqlite3.connect("Flask_Jade_Sample/TestFlaskJadeWeb/Users.db")
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        
        curr='SELECT * FROM Users WHERE username="'+str(user)+'"'
        cursor.execute(curr)
        records = cursor.fetchall()
        points = points + records[0]['Points']
        
        print([dict(row) for row in records])
        
        #update to users part
        state='UPDATE Users SET Points ='+str(points)+' WHERE username="'+str(user)+'"'
        print(state)
        cursor.execute(state)
        conn.commit()
        
        cursor.close()
        conn.close()
        return 
    
    def creatBookmarksTable():
        conn = sqlite3.connect("Flask_Jade_Sample/TestFlaskJadeWeb/Users.db")
        cursor = conn.cursor()
        table_query = """create table if not exists Bookmarks
                            (Users text, Marked integer,
                            PRIMARY KEY (Users, Marked))"""
        cursor.execute(table_query)
        conn.commit()
        cursor.close()
        conn.close()
    
    def getUserBookmarks(self, user):
        conn = sqlite3.connect("Flask_Jade_Sample/TestFlaskJadeWeb/Users.db")
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()

        select_query = 'SELECT * FROM Bookmarks WHERE User = "' + user + '"'
        cursor.execute(select_query)
        
        records = cursor.fetchall()
        
        conn.commit()
        cursor.close()
        conn.close()
        return [dict(row) for row in records]
    
    def updateBookmarks(self, user, ind):
        conn = sqlite3.connect("Flask_Jade_Sample/TestFlaskJadeWeb/Users.db")
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        
        update_query = 'INSERT OR IGNORE INTO Bookmarks (User, Marked) VALUES (?, ?)'
        cursor.execute(update_query, (user, ind,))
        
        conn.commit()
        cursor.close()
        conn.close()
        
    def updateColumn(self, user, col, val):
        conn = sqlite3.connect("Flask_Jade_Sample/TestFlaskJadeWeb/Users.db")
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        
        update_query = 'UPDATE Users SET ' + col + ' = "' + val + '" WHERE username = "' + user + '"'
        cursor.execute(state)
        conn.commit()
        
        cursor.close()
        conn.close()
        
    
    ### Indeed ###
    
    # gets a list of dictionaries limited by the input parameters through indeed
    def indeedJobs(self, job, location, maxPages):
        print('entering indeed jobs')
        baseLink = 'https://www.indeed.com/'
        webAddr = baseLink + ('' if job == '' else 'jobs?q=' + job.replace (' ', '+'))
        webAddr = webAddr + ('' if location == '' else '&l=' + location) + '&start=0'

        jbPages = []
        for x in range(0, maxPages):
            link=webAddr.replace(webAddr[-1], str(x))
            #jbPages.append(self.getDictNew(link))
            source = requests.get(link).text
            soup = BeautifulSoup(source, 'lxml')

            for div in soup.find_all ('div', class_='row', attrs={'class':'row'}):
                linkElem = div.find('div', class_='title').a
                title = linkElem.get('title') # TITLE
                link = "https://www.indeed.com" + linkElem.get('href')   # LINK

                payRAW = div.find(name='span', class_=['salaryText', 'sjcl', 'salary'])
                pay = payRAW.text.replace('\n', '') if not payRAW == None else 'N/A' # salary

                co = div.find(name='span', class_=['company', 'result-link-source']).text.strip() # company
                loc = div.find(['div', 'span'], attrs={'class': 'location'}).text # location
                date = div.find('span', attrs={'class': 'date'}).text # date

                jobMeta = self.getPageMeta(link)
                newDict = {
                    'Company':co, 'Location':loc, 'Title':title, 
                    'Time-Posted':date, 'Salary':pay, 'Link':link,
                    'Contract-Type':'N/A'
                }
                newDict['Apply-To'] = jobMeta[0]
                newDict['Skills'] = jobMeta[1]
                newDict['Desc'] = jobMeta[2]
                newDict['Page-Addr'] = jobMeta[3]
                newDict['Education'] = jobMeta[4]
                newDict['Category'] = jobMeta[5]

                jbPages.append(newDict)
        
        print('indeed jobs length ', len(jbPages))
        return jbPages
    
    # TODO: delete this
    def getDictNew(self, url):
        source = requests.get(url).text
        soup = BeautifulSoup(source, 'lxml')

        tmpLst = []
        for div in soup.find_all ('div', class_='row', attrs={'class':'row'}):
            linkElem = div.find('div', class_='title').a
            title = linkElem.get('title') # TITLE
            link = "https://www.indeed.com" + linkElem.get('href')   # LINK

            payRAW = div.find(name='span', class_=['salaryText', 'sjcl', 'salary'])
            pay = payRAW.text.replace('\n', '') if not payRAW == None else 'N/A' # salary

            co = div.find(name='span', class_=['company', 'result-link-source']).text.strip() # company
            loc = div.find(['div', 'span'], attrs={'class': 'location'}).text # location
            date = div.find('span', attrs={'class': 'date'}).text # date
            
            jobMeta = self.getPageMeta(link)
            newDict = {
                'Company':co, 'Location':loc, 'Title':title, 'Time-Posted':date, 'Salary':pay, 'Link':link
            }
            newDict['Apply-To'] = jobMeta[0]
            newDict['Skills'] = jobMeta[1]
            newDict['Desc'] = jobMeta[2]
            newDict['Page-Addr'] = jobMeta[3]
            newDict['Education'] = jobMeta[4]
            
            tmpLst.append(newDict)
        return tmpLst

In [5]:
d=data()
#d.create('science', '', 2)

In [6]:
#d.gamePoints('Mike', 10)
#d.updateBookmarks('Mike', '')
d.addCol()
#d.getUserBookmarks('Mike')

In [8]:
d.getAllUsers()

[{'userName': 'Mike',
  'passWord': 'Zyskind',
  'userType': 'Seeker',
  'Points': 230,
  'Resume': '',
  'Education': None,
  'Bookmarks': None,
  'skills': 'java,python',
  'email': 'mLz6020@gmail.com'},
 {'userName': 'Manager',
  'passWord': 'Test',
  'userType': 'Manager',
  'Points': 0,
  'Resume': '',
  'Education': None,
  'Bookmarks': None,
  'skills': None,
  'email': None},
 {'userName': 'Bob the builder',
  'passWord': 'can we do it?',
  'userType': 'Both',
  'Points': 0,
  'Resume': '',
  'Education': None,
  'Bookmarks': None,
  'skills': None,
  'email': None},
 {'userName': 'Bob the builder',
  'passWord': 'can we do it?',
  'userType': 'Both',
  'Points': 0,
  'Resume': '',
  'Education': None,
  'Bookmarks': None,
  'skills': None,
  'email': None},
 {'userName': 'manny garcia',
  'passWord': 'electroswing',
  'userType': 'seeker',
  'Points': 10,
  'Resume': '',
  'Education': None,
  'Bookmarks': None,
  'skills': None,
  'email': None},
 {'userName': 'david',
  'pas

In [3]:
#d.getUserByName('Mike')
d.destroyUsersTable()
d.creatUsersTable()

In [3]:
d.getNJobsByQuery(['science', 'chicago', 'java'], ['jobDes', 'location', 'skills'], 0, 10)

[[0,
  {'location': 'Chicago, IL',
   'company': 'Enterprise Peak',
   'datePosted': '1 day ago',
   'postUrl': 'https://www.indeed.com/company/Enterprise-Peak/jobs/Javascript-Software-Engineer-Full-Stack-Tech-Startup-9d06432fbb6f6d40?fccid=01046231f71a939f&vjs=3',
   'jobType': 'N/A',
   'jobTitle': 'JavaScript Software Engineer – Full Stack – Tech Startup',
   'jobDes': 'JavaScript Software Engineer Responsibilities: The JavaScript Software Engineer will join our Development team and be responsible for continuing development on our cutting-edge Applicant Tracking System (ATS). This role will involve both front and back-end development for new features and functionalities, including integrating disparate data-sources and improving machine-learning tools for natural language processing and automation. Opportunities will be given to own the design, architecture, and development of brand new deliverables.We are in the process of expanding our firm, which includes the development of more 

In [56]:
# converts the given list to a string
def lst_to_str(lst):
    return ','.join(lst)

# converts the given string to a list
def str_to_lst(stri):
    return stri.split(',')

In [ ]:
# CHANGES
# + updated to match commit 701d273a962d13486985e1c8852af6733eab4f77
# + updated to match commit fa30ef6af9ef9659831d2fbc5e526715340fc29e

In [7]:
conn = sqlite3.connect("Flask_Jade_Sample/TestFlaskJadeWeb/Users.db")
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())
cursor.close()
conn.close()

[('USERS',), ('JOBS',), ('Bookmarks',)]


In [3]:
d.destroyJobs()

In [74]:
len(d.getAllJobs())

265

In [7]:
d.create('computer science', '', 8)

entering github jobs
github jobs length  110
entering indeed jobs
indeed jobs length  151
create done
